# Topic Modeling in Python
In diesem Notebook wird in Kürze das Topic Modeling anhand der State-of-the-Union-Ansprachen der US-Präsidenten durchgeführt.

In [ ]:
# Einlesen des Textes
with open("state_of_the_union.txt", "r", encoding="utf-8") as file:
    text = file.read()

Wir führen ein sehr simples Preprocessing durch, um die Daten vorzubereiten. In der Datei sind Parapgraphen durch doppelte Zeilenumbrüche getrennt und wir machen es uns einfach und setzen an diesem Punkt an.

In [ ]:
texts = text.split("\n\n")

print(texts[56])

Die Texte möchten wir nun noch vorverarbeiten. Dazu gehört einerseits das *tokenisieren* also auflösen in Worteinheiten, das Entfernen von Stoppwörtern und die Lemmatisierung von Wörtern, also die Reduktion auf eine Grundform (z.B. Infinitiv).

Für das Preprocessing empfiehlt sich bei grösseren Projekten eine NLP-Bibliothek wie [spacy](https://spacy.io/). Spacy ist eine NLP-Bibliothek, die eine komplette Pipeline, u.a. auch Named-Entity- und POS-Tagging unterstützt.

Beachte, dass wir in diesem Skript numpy für eine bestimmte Version installieren, weil Spacy uns sonst Probleme macht.

In [ ]:
%pip install -U -q numpy==1.23.5 spacy

In [ ]:
# und das englische Modell
!python -m spacy download en_core_web_sm

In [ ]:
# Für Spacy müssen wir ein bestimmtes Modell herunterladen, mit dem wir die Texte vorbereiten wollen.
# Wir verwenden das kleine englische Modell, das schnell geladen ist.
# Wir schalten aber die Module ab, die wir nicht benötigen, damit die Texte schneller bearbeitet werden können.
import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [ ]:
# Anreichern der Texte mit Informationen, das dauert eine Weile. Ich beschränke hier die Anzahl Texte auf 5000, damit das Programm schneller durchläuft.
prepped_texts = []
for text in texts[:5000]:
    doc = nlp(text)
    # Wir entfernen die Stoppwörter und wandeln die Tokens in ihre Lemmata und in Kleinbuchstaben um.
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
    prepped_texts.append(lemmas)

Für das Topic Modeling verwenden wir hier [Tomotopy](https://bab2min.github.io/tomotopy/v0.13.0/en/), ein Package, das in Python sehr einfach anwendbar ist.

Wir upgrade ausserdem wir numpy, weil tomotopy eine neuere Version benötigt (unpraktisch, ja...). In einem realen Fall, wäre es am schlauesten, das Preprocessing und das Topic Modelling in diesem Fall zu trennen und verschiedene virtuelle Umgebungen zu verwenden.

In [ ]:
%pip install -U -q numpy tomotopy

In der nächsten Zelle trainieren wir das Modell und geben uns eine kurze Übersicht aus. Wir können beim Modell noch einige Parameter einstellen, die in der Dokumentation von Tomotopy näher erklärt werden. In unserem Fall wollen wir keine unheimlich seltenen Wörter beachten, aber auch die häufigsten entfernen.

In [ ]:
import tomotopy as tp

# Initialisiere das Modell, k gibt die Zahl der gewünschten Topics an
mdl = tp.LDAModel(k=10, min_df=100, rm_top=10)

# Lade alle Texte ins Modell
for text in prepped_texts:
    mdl.add_doc(text)

# Trainiere das Modell, alle 50 Trainings-Iterationen wird der Stand ausgegeben
for i in range(0, 500, 50):
    mdl.train(100)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

# Zeige, welche Topics mit welchen wichtigen Wörtern erkannt wurden
for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))

mdl.summary()

Im Viewer können wir unser Modell auf einer interaktiven Oberfläche inspizieren.

In [ ]:
tp.viewer.open_viewer(mdl, port=9999)

Wenn wir mit den Ergebnissen weiterarbeiten wollen, sollten wir sie am besten in tabellarische Form. Wir geben hier eine Tabelle aus, die Dokumente und Topics mit ihren jeweiligen Zusammenhängen zeigt. Beachte, dass hier keine Infos zu den Wort-Topic-Beziehungen gegeben sind.

In [ ]:
import csv

with open("document_topic_matrix.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # Schreibe den Header
    header = ["Document", "Text"] + [f"Topic_{i}" for i in range(mdl.k)]
    writer.writerow(header)
    
    # Schreibe die Daten
    for doc_id, doc in enumerate(mdl.docs):
        row = [f"Doc_{doc_id}", texts[doc_id]] + list(doc.get_topic_dist())
        writer.writerow(row)

print("Document-topic matrix with text written to 'document_topic_matrix.csv'")